In [1]:
import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from autogen.agentchat.contrib.qdrant_retrieve_user_proxy_agent import QdrantRetrieveUserProxyAgent


In [2]:
# zephyr-7b-beta running on local

config_list = [
    {
        "api_base": "http://localhost:8001/v1",
        "api_key": "xyz"
    }
]

# define LLM configuration
llm_config={
    "config_list": config_list,
    "temperature": 0,
    "request_timeout": 800,
    "max_retry_period": 50,
    "retry_wait_time": 1,
    "cache_seed": 42,
    "retry_wait_time": 1
}

In [3]:
# configure qdrant client
client = QdrantClient(url= 'https://3b137cea-90f8-4fea-8b72-954bf21eeed1.us-east4-0.gcp.cloud.qdrant.io:6333' , api_key= "YBM_CzRCXu_98KMmsagBUyvTK9QxkNoGJQG79D-mWywSOfEw9YlrcA")

In [5]:
# custom text splitter
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\r", "\t"])
termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

# Assistant Agent
assistant = RetrieveAssistantAgent(
    name="assistant", 
    system_message="You are a helpful assistant.",
    llm_config= llm_config
)


# Employee RAG Agent
ragproxyagent = QdrantRetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    llm_config= llm_config,
    max_consecutive_auto_reply=10,
    retrieve_config={
        "task": "qa",
        "docs_path": "/Users/anik/Desktop/code_repo/Sally_Harley_payslip.pdf",
        "custom_text_split_function": text_splitter.split_text,
        "chunk_token_size": 2000,
        "client": client,
        "embedding_model": "BAAI/bge-base-en-v1.5",
        "collection_name": "test1",
    }
)

In [6]:
assistant.reset()

qa_problem = "what is Sally Harley's date of joining?"
ragproxyagent.initiate_chat(assistant, problem=qa_problem)

Trying to create collection.
Adding doc_id 0 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: what is Sally Harley's date of joining?

Context is: Payslip  
Zoonodle  Inc 
21023 Pearson Point  Road  
Gate Avenue  
 
 
Date of Joining  : 2018 -06-23 
Pay Period  : August  2021  
Worked  Days  26 Employee  Name  : Sally  Harley  
Designation  : Marketing  Evecutive  
Department  : Marketing  
 
 
 
Earnings  
Basic Pay  Amoun t  
10000  Deductions  
Provident  Fund  Amount   
1200  
Icentive Pay   1000  Profesional Tax   500 
House Rent Allowance   400 Loan   400 
Meal Allowance   200    
 
Total 
Earnings   
11600   
Total Deductions   
2100  
  Net Pay  9500  
 
 
 
9500  
Nine Thousand